In [17]:
import pandas as pd
import numpy as np
import torch
import h5py
from torchvision.transforms import (
    Compose,
    RandomHorizontalFlip,
    RandomVerticalFlip,
)

/home/zeerak.talat/.conda/envs/featuregen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
idx = 6979

In [3]:
df = pd.read_pickle("data/more_text_IEMO.pkl")
AUDIOS = (h5py.File("data/whisper_iemo_audio.hdf5", "r", libver="latest", swmr=True))
VIDEOS = (h5py.File("data/iemo_videos_blackground.hdf5", "r", libver="latest", swmr=True))

# Checking Size

In [4]:
len(list(AUDIOS.keys())), len(list(VIDEOS.keys())), len(df)

(7380, 7380, 7380)

In [5]:
def split_len(split):    
    i = 0
    for key in AUDIOS.keys():
        if key.startswith(split):
            i += 1
    return i

def df_len(split):
    return len(df[df["split"] == split])

split_len("train"), split_len("val"), split_len("test")

(4433, 1472, 1475)

In [6]:
df_len("train"), df_len("val"), df_len("test")

(4778, 980, 1622)

# Code

In [10]:
df.iloc[idx]

text               WILLIAM: D.S.L. Extreme, can I help you? ELIZA...
index                                                 Ses05M_impro08
split                                                           test
emotion                                                            0
utterance_order                                                   14
talker                                                       WILLIAM
sentiment_label                                              neutral
dialog                                                           142
sentiment                                                          1
timings                                               (65.87, 69.61)
bbox               [[[58, 17, 186, 219]], [[59, 16, 185, 218]], [...
audio_path                 ../../data/iemocap_wav/Ses05M_impro08.wav
video_path                 ../../data/iemocap_avi/Ses05M_impro08.avi
len                                                               16
emotion_label                     

In [11]:
audio_path = df.iloc[idx]['audio_path']
video_path = df.iloc[idx]['video_path']
timings = df.iloc[idx]['timings']

## Audio

In [14]:
speech_array = torch.Tensor(AUDIOS[f"train_{audio_path.split('/')[-1][:-4]}_{timings}"][()])

In [15]:
torch.save(speech_array, f"{idx}_audio.pt")

## Video

In [18]:
transform = Compose(
    [
        RandomHorizontalFlip(p=0.5),  #
        RandomVerticalFlip(p=0.5),  #
    ]
)
video = torch.Tensor(VIDEOS[f"train_{video_path.split('/')[-1][:-4]}_{timings}"][()])  
video = transform(video)


In [19]:
torch.save(video, f"{idx}_video.pt")